### Setup

In [1]:
import polars as pl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import glob

In [2]:
pl.Config.set_tbl_rows(100)

polars.config.Config

### Load Dataframes

In [3]:
player_plays = pl.read_csv("data/player_play.csv", null_values=["NA"])
games = pl.read_csv("data/games.csv", null_values=["NA"])
plays = pl.read_csv("data/plays.csv", null_values=["NA"])
players = pl.read_csv("data/players.csv", null_values=["NA"])

In [4]:
tracking_files = glob.glob("data/tracking_week_*.csv")
tracking_data = pl.concat([pl.read_csv(f, null_values= ["NA"]) for f in tracking_files])

### Exploration

In [ ]:
player_plays.head(3)

In [ ]:
player_plays.columns

In [ ]:
games.head(2)

In [ ]:
games.columns

In [ ]:
players.head(3)

In [ ]:
players["position"].unique()

In [ ]:
plays.head(3)

In [ ]:
plays.columns

In [ ]:
tracking_data.head(3)

In [ ]:
tracking_data.columns

### Feature Engineering

In [5]:
# Rename columns in `tracking_data`
tracking_data = tracking_data.rename({"s":"yardsPerSecond", "a":"acceleration","o":"orientation"})

In [29]:
tracking_data['frameType'].unique()

frameType
str
"""BEFORE_SNAP"""
"""AFTER_SNAP"""
"""SNAP"""


##### Presnap movements 

In [6]:
presnap_movements = tracking_data.filter(
    (pl.col("frameType") == "BEFORE_SNAP")
)

In [48]:
df = tracking_data.filter(
    (pl.col("gameId") == 2022101000) & 
    (pl.col("event") == "man_in_motion")
)

In [12]:
snap = tracking_data.filter(
    (pl.col("gameId") == 2022101000) &
    (pl.col("frameType") == "SNAP")
)

In [13]:
unique_plays = snap["playId"].unique()

In [ ]:
unique_plays

In [22]:
df = df.filter(
    (pl.col("playId") == 57)
)

In [ ]:
df

In [24]:
snap = snap.filter(
    (pl.col("playId") == 57)
)

In [ ]:
snap

#### Game Analysis

In [48]:
def calculate_presnap_movement(presnap:pl.DataFrame, snap:pl.DataFrame) -> pl.DataFrame:
    merged = presnap.join(snap, on='displayName', suffix='_snap')
    merged = merged.with_columns(
     (pl.col("x")-pl.col("x_snap")).alias("x_dif"),
     (pl.col("y")-pl.col("y_snap")).alias("y_dif")
    ).sort((pl.col("y_dif")), descending=True)
    return merged

def determine_sides(game_df:pl.DataFrame, players_df:pl.DataFrame) -> pl.DataFrame:
    offensive_positions = {"QB", "RB", "T", "FB", "G", "C", "WR", "TE"}
    game_with_positions = game_df.join(players_df.select(["displayName", "position"]), on="displayName")
    game_with_sides = game_with_positions.with_columns(
        (pl.col("position").is_in(offensive_positions)).alias("isOffence")
    )
    return game_with_sides


def analyze_man_in_motion(game_df:pl.DataFrame, players_df:pl.DataFrame) -> None:
    mim_game_df = game_df.filter(
        (pl.col("event") == "man_in_motion")
    )
    plays = mim_game_df.group_by("playId")
    for play_id, play_df in plays:
        pre = play_df.filter(
            (pl.col("frameType") == "BEFORE_SNAP")
        )
        snap = game_df.filter(
            (pl.col("frameType") == "SNAP") &
            (pl.col("playId") == play_id[0])
        )
        play_side_df = determine_sides(pre, players_df)
        play_df = calculate_presnap_movement(play_side_df, snap)
        try:
            off = play_df.filter(
                (pl.col("isOffence") == True)
            ).sort((pl.col("y_dif")), descending=True)
            player= off[0]['displayName'][0]
            dis = round(off[0]['y_dif'][0], 2)
            if dis < 1:
                continue
            dir = off[0]['playDirection'][0]
            pos = off[0]['position'][0]
            print(f"Processed playId: {play_id[0]} -- {pos} {player} moved {dis} yards to the {dir}.")
        except:
            continue

In [7]:
test_game = tracking_data.filter(
    (pl.col("gameId") == 2022101000)
)

gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,yardsPerSecond,acceleration,dis,orientation,dir,event
i64,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,str
2022101000,57,41265,"""Derek Carr""",1,"""BEFORE_SNAP""","""2022-10-11 00:15:50.4""",4,"""LV""","""right""",30.79,27.19,0.01,0.02,0.05,160.57,89.24,"""huddle_break_offense"""
2022101000,57,41265,"""Derek Carr""",2,"""BEFORE_SNAP""","""2022-10-11 00:15:50.5""",4,"""LV""","""right""",30.82,27.18,0.06,1.12,0.03,139.97,108.86,null
2022101000,57,41265,"""Derek Carr""",3,"""BEFORE_SNAP""","""2022-10-11 00:15:50.6""",4,"""LV""","""right""",30.83,27.18,0.28,2.37,0.02,130.14,112.37,null
2022101000,57,41265,"""Derek Carr""",4,"""BEFORE_SNAP""","""2022-10-11 00:15:50.7""",4,"""LV""","""right""",30.88,27.16,0.61,3.0,0.05,127.5,110.94,null
2022101000,57,41265,"""Derek Carr""",5,"""BEFORE_SNAP""","""2022-10-11 00:15:50.8""",4,"""LV""","""right""",30.96,27.14,0.99,3.37,0.09,124.22,109.28,null
2022101000,57,41265,"""Derek Carr""",6,"""BEFORE_SNAP""","""2022-10-11 00:15:50.9""",4,"""LV""","""right""",31.07,27.1,1.29,2.98,0.11,122.66,108.86,null
2022101000,57,41265,"""Derek Carr""",7,"""BEFORE_SNAP""","""2022-10-11 00:15:51""",4,"""LV""","""right""",31.2,27.05,1.53,2.33,0.14,121.1,108.69,null
2022101000,57,41265,"""Derek Carr""",8,"""BEFORE_SNAP""","""2022-10-11 00:15:51.1""",4,"""LV""","""right""",31.34,27.0,1.66,1.54,0.15,118.51,108.61,null
2022101000,57,41265,"""Derek Carr""",9,"""BEFORE_SNAP""","""2022-10-11 00:15:51.2""",4,"""LV""","""right""",31.5,26.95,1.76,0.92,0.17,117.92,109.63,null


In [49]:
analyze_man_in_motion(test_game, players)

Processed playId: 2155 -- TE Travis Kelce moved 4.21 yards to the right.
Processed playId: 747 -- WR Davante Adams moved 2.05 yards to the right.
Processed playId: 57 -- WR Mack Hollins moved 17.7 yards to the right.
Processed playId: 3837 -- RB Jerick McKinnon moved 3.23 yards to the left.
Processed playId: 2071 -- WR Skyy Moore moved 15.32 yards to the right.
Processed playId: 812 -- WR Hunter Renfrow moved 14.94 yards to the right.
Processed playId: 516 -- TE Travis Kelce moved 9.44 yards to the left.
Processed playId: 2227 -- WR Mecole Hardman moved 7.76 yards to the right.
Processed playId: 2176 -- WR JuJu Smith-Schuster moved 14.67 yards to the right.
Processed playId: 224 -- WR JuJu Smith-Schuster moved 19.11 yards to the left.
Processed playId: 2631 -- WR Mecole Hardman moved 1.66 yards to the right.
